In [10]:
import tensorflow as tf
from PIL import Image
import numpy as np

In [11]:
# variables

img_size = 100

pos_num = 100
neg_num = 100

train_x = []
train_y = []

for i in range(pos_num):
    with Image.open("/Users/sungju/Dropbox/data/pos_1.jpg") as img:
        img = img.resize((img_size,img_size))
        img = np.array(list(img.getdata()))
        img = img.reshape((img_size,img_size,3))
        train_x.append(img)
        train_y.append([0, 1])

for i in range(neg_num):
    with Image.open("/Users/sungju/Dropbox/data/neg_1.jpg") as img:
        img = img.resize((img_size,img_size))
        img = np.array(list(img.getdata()))
        img = img.reshape((img_size,img_size,3))
        train_x.append(img)
        train_y.append([1, 0])

train_x = np.array(train_x)
train_y = np.array(train_y)

perm = np.random.permutation(len(train_y))

train_x = train_x[perm]
train_y = train_y[perm]


In [12]:
train_x.shape

(200, 100, 100, 3)

In [13]:
train_y.shape

(200, 2)

In [14]:
learning_rate = 0.01
epochs = 10
batch_size = 8
display_step = 10

n_channel = 3
n_classes = 2

dropout = 0.75

X = tf.placeholder(tf.float32, [None, img_size, img_size, n_channel])
Y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

In [22]:
def conv2d(X, W, b, strides=1):
    X = tf.nn.conv2d(X,W,strides=[1,strides, strides,1],padding='SAME')
    X = tf.nn.bias_add(X,b)
    return tf.nn.relu(X)

def maxpool2d(X, k =2):
    # ksize is window!, strides is just strides!
    return tf.nn.max_pool(X,ksize = [1,k,k,1],strides=[1,k,k,1], padding='SAME')

def conv_net(X, weight, biases, dropout):
    X = tf.reshape(X, shape=[-1, img_size, img_size, n_channel])
    print(X.get_shape())
    
    conv1 = conv2d(X, weights['wc1'], biases['bc1'])
    
    print(conv1.get_shape())
    conv1 = maxpool2d(conv1,k=2)
    
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)
    
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    print(weights['wd1'].get_shape().as_list()[0])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    
    fc1 = tf.nn.dropout(fc1, dropout)
    
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [23]:
weights = {
    # 5x5 conv, 1 input, 32 outputs
    # I think '1 input' means the channel's dimension is one
    'wc1': tf.Variable(tf.random_normal([5,5,3,32])),
    'wc2': tf.Variable(tf.random_normal([5,5,32,64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [24]:
pred = conv_net(X,weights, biases, keep_prob)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred,Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.initialize_all_variables()

(?, 100, 100, 3)
(?, 100, 100, 32)
3136


In [21]:
with tf.Session() as sess:
    sess.run(init)
    step = 1
    
    while step* batch_size < epochs:
        
        offset = (step * batch_size) % (train_y.shape[0] - batch_size)
        batch_x = train_x[offset:(offset + batch_size), :, :, :]
        batch_y = train_y[offset:(offset + batch_size), :]
        
        print(batch_x.shape)
        print("="*50)
        print(batch_y.shape)
        sess.run(optimizer, feed_dict={X:batch_x, Y:batch_y, keep_prob:dropout})
        
        if step% display_step == 0:
            loss, acc = sess.run([cost,accuracy], feed_dict={X:batch_x, Y:batch_y,
                                                               keep_prob: 1.})
            
            print( "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
            
        step +=1
        
    print("Optimization Finished!")
    print("Testing Accuracy"+ str(sess.run(accuracy, feed_dict={X: train_x,
                                                                    Y: train_y,
                                                                      keep_prob:1.})))

(8, 100, 100, 3)
(8, 2)


InvalidArgumentError: logits and labels must be same size: logits_size=[400,2] labels_size=[8,2]
	 [[Node: SoftmaxCrossEntropyWithLogits_2 = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](Add_5, _recv_Placeholder_4_0)]]
Caused by op 'SoftmaxCrossEntropyWithLogits_2', defined at:
  File "/usr/local/Cellar/python3/3.5.1/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python3/3.5.1/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/sungju/Dropbox/OpenSource/sj/tensorflow_practice/envTP/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/sungju/Dropbox/OpenSource/sj/tensorflow_practice/envTP/lib/python3.5/site-packages/traitlets/config/application.py", line 653, in launch_instance
    app.start()
  File